### Prep Work

In [1]:
# Import general libraries
import pandas as pd
import numpy as np

import datetime
import warnings; warnings.simplefilter('ignore')

np.random.seed(42)

In [2]:
# Increase size of dataframe
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 100000)
pd.set_option('display.width', 1000)

### Read maintenance data files

In [3]:
# Read historical maintenance data for 2015 and 2016
col_names = ['fleet', 'ac_serial_num', 'ata4_code', 'log_page_ptt_seq', 'flight_number',
             'delay_type', 'corrective_action', 'delay_code', 'date_reported',
             'station_delay_minutes', 'return_code', 'return_type_code', 'engine_pos',
             'problem_log', 'd&c_category', 'resolved_ata2_code', 'arrival_station',
             'lt_code', 'lt_minutes', 'ref_number', 'month_name', 'month_year', 'depature_station']
data1 = pd.read_excel('./data/Ds&Cs2015.xlsx', names = col_names)
data2 = pd.read_excel('./data/2016Ds&Cs.xlsx', names = col_names)

In [4]:
# Append the datasets for the 2 years
data = data1.append(data2)

# Print the shape of the combined dataset
print(f'data: {data.shape}')

# See what the data looks like
data.head(3)

data: (76437, 23)


,fleet,ac_serial_num,ata4_code,log_page_ptt_seq,flight_number,delay_type,corrective_action,delay_code,date_reported,station_delay_minutes,return_code,return_type_code,engine_pos,problem_log,d&c_category,resolved_ata2_code,arrival_station,lt_code,lt_minutes,ref_number,month_name,month_year,depature_station
0,A319-131,4001,0,NaN,278,NaN,NaN,TA,2015-01-27,46,NaN,0,NaN,NaN,Delay>15mins,0,MDT,NaN,0.0,NaN,JAN,2015-01-27,ORD
1,A319-131,4001,0,NaN,311,NaN,NaN,TA,2015-01-04,59,NaN,0,NaN,NaN,Delay>15mins,0,DEN,LT,183.0,NaN,JAN,2015-01-04,EWR
2,A319-131,4001,0,NaN,317,NaN,NaN,TA,2015-02-10,14,NaN,0,NaN,NaN,Delay 6-15mins,0,IND,NaN,0.0,NaN,FEB,2015-02-10,SFO


### Clean data

In [6]:
# Drop unnecessary columns
data.drop(columns = ['ac_serial_num', 'log_page_ptt_seq','flight_number','delay_type',
                     'date_reported','return_code','return_type_code', 'engine_pos',
                     'resolved_ata2_code','arrival_station', 'lt_code', 'lt_minutes', 
                     'ref_number', 'month_name', 'month_year', 'depature_station'], inplace = True)  

In [7]:
# Reorder the columns
data = data[['fleet','problem_log', 'corrective_action','delay_code',
             'station_delay_minutes', 'd&c_category','ata4_code']]

In [8]:
# ATA code should be an object with leading zeros
data['ata4_code'] = data['ata4_code'].astype(str).map(lambda x: x.zfill(4))

In [9]:
# Drop the rows with null values in corrective_action and problem_log
data.dropna(subset=['corrective_action', 'problem_log'], inplace=True) 

### Save full dataset before subsetting it - merge with ATA dictionary created from ATA Wikipedia

In [10]:
# Read ATA2 code descriptions from ATA Wikipedia and create dictionary
ata_all_codes = pd.read_csv('./data/ata_all_codes.csv',names=['ata4_code','ata4_desc'],
                    header=None).set_index('ata4_code')

ata_all_dict = ata_all_codes.to_dict()['ata4_desc']

# Map data to ata dictionary
data['ata4_desc'] = data['ata4_code'].astype(int).map(ata_all_dict)

# Drop the rows with null values in ata4_desc
data.dropna(subset=['ata4_desc'], inplace=True) 

# Save the full data file - all ATA codes
data.to_csv('./data/data_ata_all.csv')

### Create a dataset for ATA25 data - merge it with ATA25 data provided by the airline

In [11]:
# Select rows with ata2 code = 25+
data = data[data['ata4_code'].map(lambda x: x[:2] == '25')]

# Read ATA25 code descriptions from airline file and create dictionary
ata25_codes = pd.read_csv('./data/ata25_codes.csv',names=['ata4_code','ata4_desc'],
                    header=None).set_index('ata4_code')

ata25_dict = ata25_codes.to_dict()['ata4_desc']

# Map maintenance data to ATA25 dictionary
data['ata4_desc'] = data['ata4_code'].astype(int).map(ata25_dict)

# Drop the rows with null values in ata4_desc
data.dropna(subset=['ata4_desc'], inplace=True) 

# Save the ATA25 data file 
data.to_csv('./data/data_ata25.csv')